# Data Analyst Technical Challenge
## The Data Analyst challenge is meant to measure a candidate's skills in data analysis. During this challenge, you will be asked to:

* Review and “clean” a dataset
* Summarize the data presented by reporting average, minimum and maximum values
* Identify key trends reflected in the dataset
* Make business recommendations based on data findings

In [22]:
## Load some default Python modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline
plt.style.use('seaborn-whitegrid')

## TODO: Make sure to check all these imports 

### Question 1
First, state what data you find included in the download.
 - - - - - 
As you can see below, the included data is 952 rows of dates, pageview counts, average time spent on page for that date, the bounce rate for the day, and the count of unique pageviews.
* The `Day Index` column is a list of dates, so when importing, we'll tell Pandas to interpret it that way.
* `Pageviews` and `Unique Pageviews` are both integer columns, and we'll have to convert them to that format later (which is why the first time we use the `describe` function it doesn't give us data such as the mean).
* The `Avg. Time on Page` column is elapsed time and will need to be converted to a timedelta format.
* `Bounce Rate` is a percentage and will need to have the punctuation removed and be converted to a floating point number.

In [2]:
## Read data in pandas dataframe
dataset_df =  pd.read_csv('data-set.csv', parse_dates=['Day Index'], infer_datetime_format=True)

## List first few rows (datapoints)
dataset_df.head()

,Day Index,Pageviews,Avg. Time on Page,Bounce Rate,Unique Pageviews
0,2015-04-05,"4,981",0:03:31,79.67%,"4,174"
1,2015-04-06,"6,133",0:03:11,78.75%,"5,102"
2,2015-04-07,"5,868",0:03:30,79.85%,"4,985"
3,2015-04-08,"5,572",0:03:24,80.83%,"4,754"
4,2015-04-09,"5,581",0:03:08,80.53%,"4,714"


In [3]:
## Check statistics of the features
dataset_df.describe()

,Day Index,Pageviews,Avg. Time on Page,Bounce Rate,Unique Pageviews
count,952,952,952,952,952
unique,952,896,115,551,893
top,2016-09-08 00:00:00,"5,924",0:04:18,83.49%,"8,355"
freq,1,3,23,10,3
first,2015-04-05 00:00:00,NaN,NaN,NaN,NaN
last,2017-12-12 00:00:00,NaN,NaN,NaN,NaN


In [5]:
dataset_df['Day Index'].max() - dataset_df['Day Index'].min()

Timedelta('982 days 00:00:00')

### Question 2
You may need to “clean” the dataset. Share a brief explanation of any changes you find 
it necessary to make in order to prepare the data for analysis. State any irregularities, 
missing data, or significant assumptions you are making.
- - - - - 
## TODO: answer

Also, it seems that there are some blank rows in this dataset.  Both of these issues will be explored further later in this section.

Looking at the file in a text editor, I found some rows with missing values, but just to be sure that we caught them all, let's use `dropna`.

In [6]:
dataset_df.loc[953]

Day Index            NaT
Pageviews            NaN
Avg. Time on Page    NaT
Bounce Rate          NaN
Unique Pageviews     NaN
Name: 953, dtype: object

In [7]:
def drop_nan(df, verbose=False, how='any', inplace=True):
    if verbose:
        print("Dropping all rows with {} NaNs:".format(how))
        old_size = len(df)
        print("Old size: {}".format(old_size))

    df.dropna(how=how, axis='rows', inplace=inplace)

    if verbose:
        new_size = len(df)
        print("New size: {}".format(new_size))
        difference = old_size - new_size
        percent = (difference / old_size) * 100
        print("Dropped {} records, or {:.2f}%".format(difference, percent))
    
    return df

dataset_df = drop_nan(dataset_df, True, 'all')

Dropping all rows with all NaNs:
Old size: 954
New size: 952
Dropped 2 records, or 0.21%


Looks like there were 2 blank lines in the original csv.

Next, let's try to find those missing dates!

In [8]:
dataset_df.dtypes

Day Index             datetime64[ns]
Pageviews                     object
Avg. Time on Page    timedelta64[ns]
Bounce Rate                   object
Unique Pageviews              object
dtype: object

In [9]:
dataset_date_idx = pd.DatetimeIndex(start=dataset_df['Day Index'].min(), end=dataset_df['Day Index'].max(), freq='D')

dataset_gaps = dataset_date_idx[~dataset_date_idx.isin(dataset_df['Day Index'])]
print(dataset_gaps)

DatetimeIndex(['2015-05-01', '2015-05-02', '2015-05-03', '2015-05-04',
               '2015-05-05', '2015-05-06', '2015-05-07', '2015-05-08',
               '2015-05-09', '2015-05-10', '2015-05-11', '2015-05-12',
               '2015-05-13', '2015-05-14', '2015-05-15', '2015-05-16',
               '2015-05-17', '2015-05-18', '2015-05-19', '2015-05-20',
               '2015-05-21', '2015-05-22', '2015-05-23', '2015-05-24',
               '2015-05-25', '2015-05-26', '2015-05-27', '2015-05-28',
               '2015-05-29', '2015-05-30', '2015-05-31'],
              dtype='datetime64[ns]', freq='D')


It looks like something happened to all the data from May 2015!  Before deciding how we'll deal with these missing values, let's investigate the rest of the data

In [10]:
## 
dataset_df['Pageviews'] = dataset_df['Pageviews'].str.replace(",","")
dataset_df['Unique Pageviews'] = dataset_df['Unique Pageviews'].str.replace(",","")
dataset_df['Bounce Rate'] = dataset_df['Bounce Rate'].str.replace("%","")

##
dataset_df['Pageviews'] = pd.to_numeric(dataset_df['Pageviews'])
dataset_df['Unique Pageviews'] = pd.to_numeric(dataset_df['Unique Pageviews'])
dataset_df['Bounce Rate'] = pd.to_numeric(dataset_df['Bounce Rate'])

##
dataset_df['Pageviews'] = dataset_df['Pageviews'].astype(int)
dataset_df['Unique Pageviews'] = dataset_df['Unique Pageviews'].astype(int)
dataset_df['Bounce Rate'] *= .01

##
dataset_df['Avg. Time on Page'] = pd.to_timedelta(dataset_df['Avg. Time on Page'])

dataset_df.head()

,Day Index,Pageviews,Avg. Time on Page,Bounce Rate,Unique Pageviews
0,2015-04-05,4981,00:03:31,79.67,4174
1,2015-04-06,6133,00:03:11,78.75,5102
2,2015-04-07,5868,00:03:30,79.85,4985
3,2015-04-08,5572,00:03:24,80.83,4754
4,2015-04-09,5581,00:03:08,80.53,4714


In [16]:
dataset_df = drop_nan(dataset_df, verbose=True, how='any', inplace=False)

dataset_df.describe(include='all')

Dropping all rows with any NaNs:
Old size: 952
New size: 952
Dropped 0 records, or 0.00%


,Day Index,Pageviews,Avg. Time on Page,Bounce Rate,Unique Pageviews
count,952,952.000000,952,952.000000,952.000000
unique,952,NaN,NaN,NaN,NaN
top,2016-09-08 00:00:00,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN
first,2015-04-05 00:00:00,NaN,NaN,NaN,NaN
last,2017-12-12 00:00:00,NaN,NaN,NaN,NaN
mean,NaN,7853.911765,0 days 00:04:03.021008,0.826534,6851.549370
std,NaN,2194.618182,0 days 00:00:24.967393,0.020341,1990.279962
min,NaN,3389.000000,0 days 00:02:31,0.758000,2855.000000
25%,NaN,6099.250000,0 days 00:03:45,0.811700,5238.000000


In [ ]:
def prepare_time_features(df, drop=False):
    df['day_of_week'] = df['Day Index'].dt.weekday
    df['day_of_month'] = df['Day Index'].dt.day
    df['week'] = df['Day Index'].dt.week
    df['month'] = df['Day Index'].dt.month
    df['year'] = df['Day Index'].dt.year

    if drop:
        df.drop(columns=['Day Index'], inplace=True)
        
    return df

### Question 3
Provide a brief summary of the data. What date range is covered? What are average
and/or total values for the provided metrics? What are minimum and maximum daily
values?
- - - - - 
* This data spans the date range from April 5th, 2015 to December 12th, 2017; but seems to be missing several days as that date range spanned 982 days.

## TODO: answer

### Question 4
Provide a summary of how website traffic is generally performing. What general longterm
trends do you see in the various indicators? Provide any relevant graphs to visually
demonstrate your points.
- - - - - 
## TODO: answer

### Question 5
In terms of unique pageviews, do you see any repeating trends?
- - - - - 
## TODO: answer

### Question 6
The marketing department wants to try out placing ads on the site next year, but only
some of the time. If the goal is to show ads on days when the website will receive the
most traffic, are there specific days, weeks or months that you would recommend for or
against?
- - - - - 
## TODO: answer